### Project in progress.

In [ ]:
#For now, API below does not request for any data.

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
#from utils import load_galaxy_data

import requests
import io
import numpy as np
import os

#Loads data from url
def make_request(url):
    print("Requesting data from {}...".format(url))
    response = requests.get('https://content.codecademy.com/courses/deeplearning-with-tensorflow/'+url)
    response.raise_for_status()
    response_data = io.BytesIO(response.content)
    return response_data
    
#Loads galaxy data
def load_galaxy_data():
  
  #If cached file not found, loads data from url
  if not os.path.isfile('./cached_data.npz'):
     response_data = make_request(url='galaxydata.npz')

     with open("cached_data.npz","wb") as save_file:
      save_file.write(response_data.read())
 
  #Load data using NumPy
  data = np.load('cached_data.npz')

  print("Successfully loaded galaxy data!")
  
  return data["data"],data["labels"]

input_data, labels = load_galaxy_data()
print(input_data.shape)
print(labels.shape)

### Train-test split.

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(input_data, labels, test_size = 0.2, shuffle = True, random_state = 42, startify = labels)
#Stratifying data ensures that ratios of galaxies in my testing data will be the same as in the original dataset.

print(x_train)
print(x_valid)
print(y_train)
print(y_valid)

### Input preprocessing.

In [ ]:
#It generates batches of tensor image data with real-time data augmentation. Most of the rest parameters are left default. 

data_generator = ImageDataGenerator(rescale = 1.0/255)
#ImageDataGenerator object will normalize the pixels using the rescale parameter. (Our images are made of 255 pixels)

### Creating NumpyArrayIterators by flow method of ImageDataGenerator.

In [ ]:
#Flow is the method that takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batches indefinitely, in an infinite loop.

train_data_iterator = data_generator.flow(x_train, y_train, batch_size = 5)
valid_data_iterator = data_generator.flow(x_valid, y_valid, batch_size = 5)

print(train_data_iterator)
print(valid_data_iterator)

### Creating model.

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape = (128, 128, 3)))

#Adding 2D convolutional layer with convolution step (stride) equal to 2, processing 128x128 RGB images into tensor output.
model.add(tf.keras.layers.Conv2D(8, 3, strides = 2, activation = 'relu'))

#Adding 2D max pooling layer.
#ownsamples the input along its spatial dimensions (height and width) by taking the maximum value over an input window (of size defined by pool_size) for each channel of the input.
#The window is shifted by strides along each dimension.
model.add(tf.keras.MaxPooling2D(pool_size=(2,2), strides=(2,2)))

#Adding flattening layer. The layer flattening the input.
model.add(tf.keras.Flatten())
model.add(tf.keras.Dense(16, activation = 'relu'))

#Output layer with softmax activation function turning it's input into probability distribution.
model.add(tf.keras.Dense(4, activation = 'softmax'))


model.compile(
    optmizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = [tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()]
)

print(model.summary())

### Fitting the model.

In [ ]:
model.fit(
    train_data_interator,
    steps_per_epoch = len(x_train)/5,
    epochs = 8,
    validation_data = valid_data_iterator,
    validation_steps = len(x_valid)/5
)

### Visualizing how neural network processing images.

In [ ]:
#This code chunck loads in sample data, uses your model to make predictions, and then saves the feature maps from each convolutional layer. 
#These feature maps showcase the activations of each filter as they are convolved across the input.

from matplotlib import pyplot as plt

#Visualizes convolutional layer activations
def visualize_activations(model, valid_iterator):

  #A keras model that will output our previous model's activations for each convolutional layer:
  activation_extractor = tf.keras.Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers if "conv2d" in layer.name])

  #Take matplotlib frame and remove axes.
  def clean_plot(plot):
    plot.axes.get_xaxis().set_visible(False)
    plot.axes.get_yaxis().set_visible(False)

  #Dict mapping from class numbers to string labels:
  class_names = {0:"Regular",1:"Ringed",2:"Merger",3:"Other"}

  #Loads a sample batch of data
  sample_batch_input,sample_labels = validation_iterator.next()
 
  #Grabs the first five images
  sample_batch_input = sample_batch_input[:5]
  sample_labels = sample_labels[:5]

  #Makes predictions using model.predict(x)
  sample_predictions = model.predict(sample_batch_input)

  #Iterate of images, predictions, and true labels
  for i,(image, prediction, label) in enumerate(zip(sample_batch_input, sample_predictions, sample_labels)):

    image_name = "Galaxy_{}".format(i)

    #Gets predicted class with highest probability

    predicted_class = tf.argmax(prediction).numpy()

    #Gets correct label
    actual_class = tf.argmax(label).numpy()

    print(image_name)
    print("\tModel prediction: {}".format(prediction))
    print("\tTrue label: {} ({})".format(class_names[actual_class], actual_class))
    print("\tCorrect:", predicted_class == actual_class)

    #Saves image file using matplotlib
    sample_image = image
    clean_plot(plt.imshow(sample_image))

    plt.title(image_name+" Predicted: {}, Actual: {}".format(class_names[predicted_class], class_names[actual_class]))
    plt.savefig('static/images/'+image_name+".png")
    model_layer_output = activation_extractor(tf.expand_dims(sample_image,0))
    
    plt.clf()

    #Iterates over each layer output
    for l_num,output_data in enumerate(model_layer_output):

      #Creates a subplot for each filter
      fig, axs = plt.subplots(1, output_data.shape[-1])
      
      #For each filter
      for i in range(output_data.shape[-1]):

        #Plots the filter's activations
        
        clean_plot(axs[i].imshow(output_data[0][:, :, i], cmap="gray"))
      plt.suptitle(image_name+" Conv {}".format(l_num),y=0.6)
      plt.savefig('static/images/' + image_name+ "Conv{}.png".format(l_num))
      plt.clf()
    
visualize_activations(model,training_data_interator)